In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.datasets import load_iris
from sklearn.ensemble import GradientBoostingClassifier,ExtraTreesClassifier,AdaBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import StackingClassifier, RandomForestClassifier



In [3]:
df = pd.read_csv("C:\\Users\Ali\\DataScience_Projects\\Kaggle_Competition\\train_set.csv")
df.head()

,RecordId,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X70,X71,X72,X73,X74,X75,X76,X77,X78,Y
0,1,87.000000,34.118411,0,2,0,165.100000,1,829,2,...,0.040000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2,82.372284,31.573280,0,0,1,162.983897,1,724,0,...,0.033431,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,3,50.000000,27.771653,0,0,1,165.100000,1,895,2,...,0.010000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,4,66.236109,26.515922,0,0,1,167.009549,1,637,0,...,0.039363,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,5,81.303299,20.843691,0,0,1,158.165419,0,564,0,...,0.069242,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [4]:
imr = SimpleImputer(missing_values=np.nan, strategy='mean')
imr = imr.fit(df.values)
imputed_data = imr.transform(df.values)
df= pd.DataFrame(imputed_data)

In [6]:
# Separating the target variable
X = df.values[:, 1:78] # 1 - 78 is 77 features... (78 is exclusive)
Y = df.values[:, 78]

# Splitting the dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.15, random_state=42)

In [7]:
gd_model = GradientBoostingClassifier(n_estimators=150, learning_rate=0.1, max_depth=3, random_state=42)
gd_model.fit(X_train, y_train)

GradientBoostingClassifier(n_estimators=150, random_state=42)

In [8]:
ad_model = AdaBoostClassifier(n_estimators=100,learning_rate=0.1)
ad_model.fit(X_train, y_train)

AdaBoostClassifier(learning_rate=0.1, n_estimators=100)

In [9]:
rf_model = RandomForestClassifier(n_estimators=150,bootstrap=True, random_state=100,max_depth=5)
rf_model.fit(X_train, y_train)

RandomForestClassifier(max_depth=5, n_estimators=150, random_state=100)

In [10]:
dt_model = DecisionTreeClassifier(max_depth=7, min_samples_split=100)
dt_model.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=7, min_samples_split=100)

In [12]:
estimators = [('Decision Tree', dt_model),
                ('GraidentBoost', gd_model),
                ('Adaboost', ad_model),
                ('Random Forest', rf_model),
                ('Extra Trees', ExtraTreesClassifier(n_estimators=100, bootstrap=True))]

st_clf = StackingClassifier(estimators=estimators, final_estimator=RandomForestClassifier())
st_clf.fit(X_train, y_train)

KeyboardInterrupt: 

In [13]:
md_predictions_probs = st_clf.predict_proba(X_test)
md_predictions_probs 

NotFittedError: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
accuracy = accuracy_score(y_test,md_predictions_probs)

In [ ]:
test_data = pd.read_csv("C:\\Users\Ali\\DataScience_Projects\\Kaggle_Competition\\test_set.csv")

In [ ]:
recordId = test_data['RecordId']
test_data = test_data.drop(columns = 'RecordId')

In [ ]:
imr = SimpleImputer(missing_values=np.nan, strategy='mean')
imr = imr.fit(test_data)
imputed_data = imr.transform(test_data)
test_data = pd.DataFrame(imputed_data)

In [ ]:
prediction = st_clf.predict_proba(test_data)
prediction= prediction[:,1]

In [ ]:
finaldf = pd.concat([recordId, pd.DataFrame(prediction, columns=['Y'])], axis=1)
finaldf
finaldf.to_csv("C:\\Users\Ali\\DataScience_Projects\\Kaggle_Competition\\Stacking_test_results.csv", index= False)